In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import json

In [2]:
base_url = 'https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/global-shark-attack/records?limit=100'

In [6]:
requests.get(base_url)

<Response [200]>

In [7]:
#request = (requests.get(base_url).json())

In [3]:
total_count = requests.get(base_url).json()['total_count']

In [4]:
#hold the data
all_of_the_data = []

In [5]:
for currentOffset in np.arange(0,total_count,100):
  # print(currentOffset)
  cur_data = requests.get(f'{base_url}&offset={currentOffset}').json()['results']
  all_of_the_data.extend(cur_data) #extend is like
  time.sleep(0.5)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800


In [12]:
len(all_of_the_data)

6890

In [13]:
pd.DataFrame(all_of_the_data)

,date,year,type,country,area,location,activity,name,sex,age,...,fatal_y_n,time,species,investigator_or_source,pdf,href_formula,href,case_number,case_number0,original_order
0,2023-05-13,2023,Unprovoked,AUSTRALIA,South Australia,Elliston,Surfing,Simon Baccanello,M,46,...,Y,10h10,White shark,"B. Myatt & S. De Marchi, GSAF",None,None,None,None,None,None
1,2023-04-29,2023,Unprovoked,AUSTRALIA,Western Australia,"Yallingup, Busselton",Swimming,male,M,None,...,N,11h20,1m shark,"The West Austalian, 4/30/2023",None,None,None,None,None,None
2,2022-10-07,2022,Unprovoked,AUSTRALIA,Western Australia,Port Hedland,Spearfishing,Robbie Peck,M,38,...,N,11h30,Bull shark,"ABC,net, 10/07/2022",None,None,None,None,None,None
3,2021-10-04,2021,Unprovoked,USA,Florida,"Fort Pierce State Park, St. Lucie County",Surfing,Truman Van Patrick,M,25.0,...,N,None,None,"K. McMurray, TrackingSharks.com",2021.10.04-VanPatrick.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2021.10.04,2021.10.04,6711
4,2021-10-03,2021,Unprovoked,USA,Florida,"Jensen Beach, Martin County",Swimming,male,M,None,...,N,12h00,None,"CBS12, 10/3/2021",2021.10.03.b-JensenBeach.pdf1,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2021.10.03.b,2021.10.04.b,6710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6885,None,None,Sea Disaster,GREECE,Off Thessaly,None,Shipwrecked Persian Fleet,males,M,None,...,Y,None,None,Herodotus (485 - 425 B.C.),493BC-PersianFleet.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,493BC-PersianFleet,493BC,128
6886,None,None,Unprovoked,INDONESIA,Riau Province,"Natuna Islands, between Sumatra & Kalimantan i...",Swimming near anchored ship,a ship's engineer,M,None,...,Y,None,None,"C.H. Townsend, p. 172; V.M. Coppleson, p.258",ND-0054-NatunaIslands.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0054,ND.0054,52
6887,None,None,Unprovoked,IRAQ,Basrah,Shatt-el Arab River near a small boat stand,Swimming,male,M,13 or 14,...,Y,Afternoon,Bull shark,B.W. Coad & L.A.J. Al-Hassan,ND-0038-Shatt-al-Arab.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0038,ND.0038,38
6888,None,None,Unprovoked,SOUTH AFRICA,KwaZulu-Natal,Durban,Crew swimming alongside their anchored ship,male,M,None,...,Y,None,None,"Captain A. Anderson, Natal Mercury, 12/31/192...",ND-0017-alongside-ship.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0017,ND.0017,18
